# Sinusoidal Neural Field Interpolation Evaluation

This notebook evaluates PixNerd's neural field interpolation capabilities for super-resolution.

## Benchmark Design

- **Training**: Model sees regularly sampled pixels (simulating low-res input)
- **Testing**: Model must predict ALL pixels (super-resolution)
- **Ground Truth**: Known sinusoidal patterns (smooth, continuous)

### Super-Resolution Simulation
Regular grid sampling simulates a downsampled low-resolution input:
- `downsample_factor=4`: Every 4th pixel visible = 4x super-res (6.25% visible in 2D)
- `downsample_factor=2`: Every 2nd pixel visible = 2x super-res (25% visible in 2D)

### Key Metrics
1. **Visible MSE**: Error on sampled positions (training data)
2. **Invisible MSE**: Error on unseen positions (interpolation quality)
3. **Interpolation Ratio**: invisible_mse / visible_mse (lower = better)

In [ ]:
import os
import sys
from pathlib import Path

# Setup paths
NOTEBOOK_DIR = Path.cwd()
PIXNERD_DIR = NOTEBOOK_DIR / "PixNerd"

if PIXNERD_DIR.exists():
    os.chdir(PIXNERD_DIR)
    sys.path.insert(0, str(PIXNERD_DIR))
    print(f"Working directory: {os.getcwd()}")
else:
    print(f"ERROR: PixNerd directory not found at {PIXNERD_DIR}")

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

torch._dynamo.config.disable = True

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Configuration

In [ ]:
# =============================================================================
# CONFIGURATION - Must match training!
# =============================================================================

CHECKPOINT_PATH = str(NOTEBOOK_DIR / "workdirs/exp_sinusoidal_nf_test/checkpoints/last.ckpt")

# Dataset config
RESOLUTION = 64  # High-res target
CHANNELS = 1
NUM_COMPONENTS = 5

# Super-resolution config (must match training)
DOWNSAMPLE_FACTOR = 4  # 4x super-res: every 4th pixel visible
MASK_MODE = "grid"  # 2D grid sampling

# Model config
PATCH_SIZE = 4
HIDDEN_SIZE = 256
DECODER_HIDDEN_SIZE = 64
NUM_ENCODER_BLOCKS = 6
NUM_DECODER_BLOCKS = 2
NUM_GROUPS = 4

# Sampling
NUM_SAMPLE_STEPS = 50

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Compute visible ratio
LOW_RES = RESOLUTION // DOWNSAMPLE_FACTOR
if MASK_MODE == "grid":
    VISIBLE_RATIO = 1.0 / (DOWNSAMPLE_FACTOR ** 2)
else:
    VISIBLE_RATIO = 1.0 / DOWNSAMPLE_FACTOR

print(f"Checkpoint: {CHECKPOINT_PATH}")
print(f"High-res target: {RESOLUTION}x{RESOLUTION}")
print(f"Low-res input: {LOW_RES}x{LOW_RES}")
print(f"Super-resolution: {DOWNSAMPLE_FACTOR}x")
print(f"Visible ratio: {VISIBLE_RATIO:.1%}")

## Load Model

In [ ]:
from src.models.autoencoder.pixel import PixelAE
from src.models.transformer.pixnerd_c2i_heavydecoder import PixNerDiT
from src.diffusion.flow_matching.scheduling import LinearScheduler
from src.diffusion.flow_matching.sampling import EulerSampler, ode_step_fn
from src.diffusion.base.guidance import simple_guidance_fn

# Import dataset utilities
sys.path.insert(0, str(NOTEBOOK_DIR))
from train_sinusoidal import SinusoidalPLModule, SinusoidalLightningModel, SimpleEMA
from train_sinusoidal import UnconditionalConditioner, MaskedFlowMatchingTrainer
from PixNerd.src.data.dataset.sinusoidal import (
    generate_sinusoidal_image,
    create_visibility_mask,
    compute_interpolation_metrics,
)


def build_model():
    scheduler = LinearScheduler()
    vae = PixelAE(scale=1.0)
    conditioner = UnconditionalConditioner()
    
    denoiser = PixNerDiT(
        in_channels=CHANNELS,
        patch_size=PATCH_SIZE,
        num_groups=NUM_GROUPS,
        hidden_size=HIDDEN_SIZE,
        decoder_hidden_size=DECODER_HIDDEN_SIZE,
        num_encoder_blocks=NUM_ENCODER_BLOCKS,
        num_decoder_blocks=NUM_DECODER_BLOCKS,
        num_classes=1,
    )
    
    trainer = MaskedFlowMatchingTrainer(
        scheduler=scheduler,
        lognorm_t=True,
        timeshift=1.0,
    )
    
    sampler = EulerSampler(
        num_steps=NUM_SAMPLE_STEPS,
        guidance=1.0,
        guidance_interval_min=0.0,
        guidance_interval_max=1.0,
        scheduler=scheduler,
        w_scheduler=LinearScheduler(),
        guidance_fn=simple_guidance_fn,
        step_fn=ode_step_fn,
    )
    
    ema_tracker = SimpleEMA(decay=0.9999)
    
    model = SinusoidalLightningModel(
        vae=vae,
        conditioner=conditioner,
        denoiser=denoiser,
        diffusion_trainer=trainer,
        diffusion_sampler=sampler,
        ema_tracker=ema_tracker,
        optimizer=None,
    )
    
    return model


print("Building model...")
model = build_model()
model.setup_ema()

print(f"\nLoading checkpoint: {CHECKPOINT_PATH}")
if os.path.exists(CHECKPOINT_PATH):
    checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
    # Extract model state from Lightning checkpoint
    state_dict = checkpoint.get('state_dict', checkpoint)
    # Remove 'model.' prefix if present
    new_state_dict = {}
    for k, v in state_dict.items():
        if k.startswith('model.'):
            new_state_dict[k[6:]] = v
        else:
            new_state_dict[k] = v
    model.load_state_dict(new_state_dict, strict=False)
    print("Checkpoint loaded!")
else:
    print(f"WARNING: Checkpoint not found!")

model = model.to(DEVICE)
model.eval()

print(f"\nTotal parameters: {sum(p.numel() for p in model.denoiser.parameters()):,}")

## Create Visibility Mask

In [ ]:
# Create visibility mask (regular grid sampling)
visibility_mask = create_visibility_mask(
    RESOLUTION, DOWNSAMPLE_FACTOR, MASK_MODE
)

visible_ratio = visibility_mask.sum() / visibility_mask.size

print(f"Visibility mask shape: {visibility_mask.shape}")
print(f"Visible pixels: {visibility_mask.sum()} / {visibility_mask.size} ({visible_ratio:.1%})")
print(f"This simulates {LOW_RES}x{LOW_RES} → {RESOLUTION}x{RESOLUTION} super-resolution")

# Visualize mask
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Full mask
axes[0].imshow(visibility_mask, cmap='RdYlGn', interpolation='nearest')
axes[0].set_title(f"Visibility Mask ({visible_ratio:.1%} visible = green)")

# Zoomed view to show grid pattern
zoom_size = 16
axes[1].imshow(visibility_mask[:zoom_size, :zoom_size], cmap='RdYlGn', interpolation='nearest')
axes[1].set_title(f"Zoomed View (top-left {zoom_size}x{zoom_size})")
for ax in axes:
    ax.set_xlabel("x")
    ax.set_ylabel("y")

plt.tight_layout()
plt.show()

print(f"\nGrid pattern: Every {DOWNSAMPLE_FACTOR}th pixel in both x and y directions")

## Generate Ground Truth Samples

In [ ]:
def generate_test_samples(num_samples=10, seed_start=1000):
    """Generate ground truth sinusoidal images for testing."""
    samples = []
    for i in range(num_samples):
        img = generate_sinusoidal_image(
            RESOLUTION, NUM_COMPONENTS,
            freq_range=(1.0, 8.0),
            seed=seed_start + i
        )
        samples.append(img)
    return np.stack(samples)


# Generate test samples
test_samples = generate_test_samples(10)
print(f"Generated {len(test_samples)} test samples")

# Display a few
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for i, (ax, img) in enumerate(zip(axes.flat, test_samples)):
    ax.imshow(img, cmap='viridis')
    ax.set_title(f"Sample {i}")
    ax.axis('off')
plt.suptitle("Ground Truth Sinusoidal Patterns")
plt.tight_layout()
plt.show()

## Reconstruction Test

Test 1: Given a noisy version of a sinusoidal image, can the model reconstruct it?

The model was trained only on visible regions, but must predict ALL pixels.

In [ ]:
@torch.no_grad()
def reconstruct_from_noise(model, num_samples=4):
    """Generate samples from pure noise."""
    model.eval()
    
    # Random noise
    noise = torch.randn(num_samples, CHANNELS, RESOLUTION, RESOLUTION, device=DEVICE)
    
    # Get condition (dummy)
    condition, uncondition = model.conditioner(noise)
    
    # Sample using EMA model
    denoiser = model.ema_denoiser if model.ema_denoiser is not None else model.denoiser
    
    samples = model.diffusion_sampler(
        denoiser,
        noise,
        condition,
        uncondition,
    )
    
    # Decode
    images = model.vae.decode(samples)
    
    return images.clamp(0, 1).cpu().numpy()


# Generate samples
print("Generating samples from noise...")
generated = reconstruct_from_noise(model, num_samples=8)

# Display
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
for i, (ax, img) in enumerate(zip(axes.flat, generated)):
    ax.imshow(img[0], cmap='viridis')  # [C, H, W] -> [H, W]
    ax.set_title(f"Generated {i}")
    ax.axis('off')
plt.suptitle("Generated Samples from Noise")
plt.tight_layout()
plt.show()

## Interpolation Quality Analysis

Key question: How well does the model predict UNSEEN regions?

We compare:
1. MSE on visible regions (training data)
2. MSE on invisible regions (interpolation quality)

In [ ]:
def visualize_interpolation(ground_truth, generated, mask):
    """
    Visualize interpolation quality.
    
    Shows:
    - Ground truth
    - Generated image
    - Error map
    - Visible/invisible region errors
    """
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # Ground truth
    axes[0, 0].imshow(ground_truth, cmap='viridis')
    axes[0, 0].set_title("Ground Truth")
    axes[0, 0].axis('off')
    
    # Generated
    axes[0, 1].imshow(generated, cmap='viridis')
    axes[0, 1].set_title("Generated")
    axes[0, 1].axis('off')
    
    # Error map
    error = np.abs(ground_truth - generated)
    im = axes[0, 2].imshow(error, cmap='hot', vmin=0, vmax=0.5)
    axes[0, 2].set_title("Absolute Error")
    axes[0, 2].axis('off')
    plt.colorbar(im, ax=axes[0, 2])
    
    # Visibility mask overlay
    overlay = np.stack([ground_truth, ground_truth, ground_truth], axis=-1)
    overlay[~mask] = [1, 0, 0]  # Red for invisible regions
    axes[1, 0].imshow(overlay * 0.5 + 0.5 * ground_truth[..., np.newaxis])
    axes[1, 0].set_title("Visibility Mask (red = unseen)")
    axes[1, 0].axis('off')
    
    # Error on visible regions only
    visible_error = error.copy()
    visible_error[~mask] = 0
    im = axes[1, 1].imshow(visible_error, cmap='hot', vmin=0, vmax=0.5)
    visible_mse = (error[mask] ** 2).mean()
    axes[1, 1].set_title(f"Error on VISIBLE ({visible_mse:.4f} MSE)")
    axes[1, 1].axis('off')
    
    # Error on invisible regions only
    invisible_error = error.copy()
    invisible_error[mask] = 0
    im = axes[1, 2].imshow(invisible_error, cmap='hot', vmin=0, vmax=0.5)
    invisible_mse = (error[~mask] ** 2).mean()
    axes[1, 2].set_title(f"Error on INVISIBLE ({invisible_mse:.4f} MSE)")
    axes[1, 2].axis('off')
    
    ratio = invisible_mse / (visible_mse + 1e-8)
    plt.suptitle(f"Interpolation Quality - Ratio: {ratio:.2f}x (lower = better)", fontsize=14)
    plt.tight_layout()
    plt.show()
    
    return visible_mse, invisible_mse, ratio

In [ ]:
# Analyze interpolation quality on generated samples
# Visualize the generated samples with the grid mask overlay

for i in range(min(4, len(generated))):
    gen_img = generated[i, 0]  # [H, W]
    
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    
    # Generated image
    axes[0].imshow(gen_img, cmap='viridis')
    axes[0].set_title("Generated")
    axes[0].axis('off')
    
    # With mask overlay (show sampled grid points)
    overlay = gen_img.copy()
    axes[1].imshow(overlay, cmap='viridis')
    # Mark visible (sampled) positions
    y_vis, x_vis = np.where(visibility_mask)
    axes[1].scatter(x_vis, y_vis, c='red', s=2, alpha=0.5)
    axes[1].set_title(f"Sampled positions (red dots, {DOWNSAMPLE_FACTOR}x grid)")
    axes[1].axis('off')
    
    # Gradient magnitude (smoothness check)
    grad_x = np.abs(np.diff(gen_img, axis=1))
    grad_y = np.abs(np.diff(gen_img, axis=0))
    axes[2].imshow(grad_x[:-1, :], cmap='hot')
    axes[2].set_title("Horizontal Gradient (edges)")
    axes[2].axis('off')
    
    plt.suptitle(f"Sample {i} Analysis")
    plt.tight_layout()
    plt.show()

## Boundary Analysis

Check if there are visible artifacts at the boundaries between visible and invisible regions.

In [ ]:
def analyze_boundaries(generated, mask):
    """
    Analyze gradient magnitude at visibility boundaries.
    
    If interpolation is poor, we expect high gradients at boundaries.
    """
    results = []
    
    for img in generated:
        img = img[0]  # [H, W]
        
        # Compute gradient
        grad_x = np.abs(np.diff(img, axis=1))
        grad_x = np.pad(grad_x, ((0, 0), (0, 1)), mode='edge')
        
        # Find boundary pixels (where mask changes)
        mask_diff = np.abs(np.diff(mask.astype(float), axis=1))
        mask_diff = np.pad(mask_diff, ((0, 0), (0, 1)), mode='constant')
        boundary_mask = mask_diff > 0
        
        # Gradient at boundaries vs elsewhere
        grad_at_boundary = grad_x[boundary_mask].mean() if boundary_mask.any() else 0
        grad_elsewhere = grad_x[~boundary_mask].mean()
        
        results.append({
            'grad_at_boundary': grad_at_boundary,
            'grad_elsewhere': grad_elsewhere,
            'ratio': grad_at_boundary / (grad_elsewhere + 1e-8)
        })
    
    return results


# Analyze boundaries
boundary_results = analyze_boundaries(generated, visibility_mask)

print("Boundary Analysis:")
print("(Ratio close to 1.0 = good interpolation, no boundary artifacts)")
print()
for i, r in enumerate(boundary_results):
    print(f"Sample {i}: boundary_grad={r['grad_at_boundary']:.4f}, "
          f"other_grad={r['grad_elsewhere']:.4f}, ratio={r['ratio']:.2f}")

avg_ratio = np.mean([r['ratio'] for r in boundary_results])
print(f"\nAverage boundary ratio: {avg_ratio:.2f}")

## Summary

This benchmark tests neural field interpolation for super-resolution:

1. **Training** on regularly sampled grid (simulating low-res input)
2. **Evaluating** reconstruction quality on unseen positions
3. **Checking** for artifacts in interpolated regions

### Super-Resolution Setup
- `downsample_factor=4`: 4x super-res (16x16 → 64x64)
- Regular grid sampling (every 4th pixel in x and y)
- ~6.25% visible pixels (256/4096)

### Key Metrics:
- **Interpolation ratio** < 2.0: Good interpolation
- **Boundary ratio** ≈ 1.0: No grid artifacts
- **Visual smoothness**: Generated patterns should be smooth sinusoids

### Next Steps:
- Try different downsample factors (2, 4, 8)
- Test column-only or row-only sampling
- Compare different model architectures